In [12]:
#!/usr/bin/env python# coding: utf-8
# ==========================================
# 📘 PSV Unified Data Cleaning & Preparation
# ==========================================
# Author: Georgi
# Description:
# - Cleans and standardizes PSV-related CSV datasets
# - Auto-detects and fixes nested JSON/list fields
# - Converts percentage and count fields to numeric
# - Handles both structured & unstructured data
# - Correctly parses semicolon-delimited text with quotes
# - Outputs all cleaned CSVs into /cleaned_final/
# ==========================================


import pandas as pd
import numpy as np
import os
import json
from ast import literal_eval
import csv
import tkinter as tk
from tkinter import filedialog


In [13]:
# -----------------------------
# Setup
# -----------------------------
import tkinter as tk
from tkinter import filedialog

# Ask user to select folder with CSVs
root = tk.Tk()
root.withdraw()  # Hide the small tkinter window

print("📂 Please select the folder containing your PSV CSV files...")
data_folder = filedialog.askdirectory(title="Select Folder with PSV CSV Files")

if not data_folder:
    raise SystemExit("❌ No folder selected. Exiting notebook.")

print(f"✅ Selected folder: {data_folder}\n")

# Create output folder in same directory
output_folder = os.path.join(data_folder, "cleaned_final")
os.makedirs(output_folder, exist_ok=True)

datasets = {
    "transfermarket": {"file": "Transfermarket.csv", "sep": ","},
    "competitor_profiles": {"file": "combined_competitor_profiles.csv", "sep": ","},
    "news_topics": {"file": "news_topics-sentiment-actions.csv", "sep": ","},
    "socials_topics": {"file": "socials_topics-sentiment-actions.csv", "sep": ","},
    "socials_overview": {"file": "socials-posts-overview.csv", "sep": ","},
    "google_headlines": {"file": "Google_headlines.csv", "sep": ";"},
    "eventregistry_headlines": {"file": "Eventregistry_headlines.csv", "sep": ","},
    "facebook_comments": {"file": "facebook-comments.csv", "sep": ","},
    "facebook_competitors": {"file": "facebook-competitor-profiles.csv", "sep": ";"},
    "facebook_posts": {"file": "facebook-posts_overview.csv", "sep": ","},
    "instagram_comments": {"file": "instagram-comments.csv", "sep": ","},
    "instagram_posts": {"file": "instagram-posts_overview.csv", "sep": ","},
    "tiktok_comments": {"file": "tiktok-comments.csv", "sep": ","},
    "tiktok_posts": {"file": "tiktok-posts_overview.csv", "sep": ","},
    "youtube_comments": {"file": "youtube-comments.csv", "sep": ";"},
    "youtube_posts": {"file": "youtube-posts.csv", "sep": ";"},
    "youtube_overview": {"file": "youtube-posts-overview.csv", "sep": ";"},
    "youtube_competitors": {"file": "youtube-competitor-profiles.csv", "sep": ";"},
    "tiktok_competitors": {"file": "tiktok-competitor-profiles.csv", "sep": ";"},
    "instagram_competitors": {"file": "instagram-competitor-profiles.csv", "sep": ";"},
    "twitter_competitors": {"file": "twitter-competitor-profiles.csv", "sep": ";"},
    "psv_matches": {"file": "psv_matches_season_25_26.csv", "sep": ","},
}


📂 Please select the folder containing your PSV CSV files...
✅ Selected folder: C:/Users/georg/Downloads/testings/working_files_to_cleaned



In [17]:
# -----------------------------
# Helper Functions
# -----------------------------
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Generic cleanup for all datasets."""
    df.dropna(how="all", inplace=True)
    df.drop_duplicates(inplace=True)
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .replace({"not available": np.nan, "N/A": np.nan, "": np.nan})
        )
    return df.reset_index(drop=True)

def try_parse_json(value):
    """Safely parse JSON-like text fields."""
    if not isinstance(value, str):
        return value
    value = value.strip()
    if value in ["", "nan", "None"]:
        return np.nan
    try:
        return json.loads(value)
    except:
        try:
            return literal_eval(value)
        except:
            return value

def convert_numeric(df: pd.DataFrame):
    """Convert percentage and numeric count columns."""
    for col in df.columns:
        if "%" in col or "rate" in col or "count" in col or "score" in col:
            df[col] = pd.to_numeric(df[col], errors="ignore")
    return df

def load_and_clean_csv(file_path: str, sep: str, dataset_name: str) -> pd.DataFrame:
    try:
        # Add quote handling for tricky files
        df = pd.read_csv(
            file_path,
            sep=sep,
            engine="c",
            on_bad_lines="skip",
            dtype=str,
            low_memory=False,
            quoting=csv.QUOTE_MINIMAL,
        )

        df = clean_dataframe(df)

        # Dataset-specific logic
        # --------------------------------

        # Competitor normalization
        if "competitor_profiles" in dataset_name and "club" in df.columns:
            df["club"] = df["club"].str.upper().str.strip()
            club_replacements = {
                "FEYENOORD ROTTERDAM": "FEYENOORD",
                "PSV EINDHOVEN": "PSV",
                "AJAX AMSTERDAM": "AJAX",
                "AZ ALKMAAR": "AZ",
                "FC TWENTE ENSCHEDE": "FC TWENTE",
            }
            df["club"] = df["club"].replace(club_replacements)
            if {"followers", "source"}.issubset(df.columns):
                df = df.groupby(["club", "source"], as_index=False)["followers"].sum()

        # Twitter competitor posts (already structured JSON)
        elif "twitter_competitors" in dataset_name:
            all_posts = []
            for _, row in df.iterrows():
                posts_raw = row.get("posts", None)
                if not posts_raw or posts_raw.strip() == "" or posts_raw == "[]":
                    continue
                try:
                    posts_list = json.loads(posts_raw)
                except:
                    try:
                        posts_list = literal_eval(posts_raw)
                    except:
                        continue
                for post in posts_list:
                    all_posts.append(
                        {
                            "profile_id": row.get("id"),
                            "profile_name": row.get("profile_name"),
                            "followers": row.get("followers"),
                            "post_id": post.get("post_id"),
                            "description": post.get("description"),
                            "date_posted": post.get("date_posted"),
                            "likes": post.get("likes"),
                            "views": post.get("views"),
                            "reposts": post.get("reposts"),
                            "replies": post.get("replies"),
                            "hashtags": post.get("hashtags"),
                        }
                    )
            df = pd.DataFrame(all_posts)

        # Instagram competitor posts (JSON-like structure)
        elif "instagram_competitors" in dataset_name and "posts" in df.columns:
            all_posts = []
            for _, row in df.iterrows():
                posts_raw = row["posts"]
                if pd.isna(posts_raw) or posts_raw.strip() == "":
                    continue
                posts_list = try_parse_json(posts_raw)
                if not isinstance(posts_list, list):
                    continue
                for post in posts_list:
                    all_posts.append(
                        {
                            "account": row.get("account"),
                            "followers": row.get("followers"),
                            "post_id": post.get("id"),
                            "caption": post.get("caption"),
                            "likes": post.get("likes"),
                            "comments": post.get("comments"),
                            "date_posted": post.get("date_posted"),
                            "hashtags": post.get("hashtags"),
                        }
                    )
            if all_posts:
                df = pd.DataFrame(all_posts)

        # TikTok competitor profiles (parse top_videos, top_posts_data)
        elif "tiktok_competitors" in dataset_name:
            for field in ["top_videos", "top_posts_data", "discovery_input"]:
                if field in df.columns:
                    df[field] = df[field].apply(try_parse_json)

        # Posts overview (hashtags list parsing)
        elif "posts_overview" in dataset_name and "hashtags" in df.columns:
            df["hashtags"] = df["hashtags"].apply(try_parse_json)

        # Sentiment-based datasets (convert % to floats)
        elif "news_topics" in dataset_name or "socials_topics" in dataset_name:
            df = convert_numeric(df)

        # Facebook competitor quote issues handled via quoting above

        # Convert all numeric fields possible
        df = convert_numeric(df)

        # Fill missing columns and clean
        df = df.reindex(columns=sorted(df.columns))
        df = df.replace("nan", np.nan)

        print(f"✅ {dataset_name}: {len(df)} rows, {len(df.columns)} cols cleaned successfully.")
        return df

    except Exception as e:
        print(f"❌ {dataset_name} failed: {e}")
        return pd.DataFrame()

In [18]:
# -----------------------------
# Process all datasets
# -----------------------------
cleaned_dataframes = {}

for name, meta in datasets.items():

    sep = meta["sep"]
    path = os.path.join(data_folder, meta["file"])
    if not os.path.exists(path):
        print(f"⚠️ Missing file: {path}")
        continue

    cleaned_df = load_and_clean_csv(path, sep, name)
    cleaned_dataframes[name] = cleaned_df

    out_path = os.path.join(output_folder, f"{name}_cleaned.csv")
    cleaned_df.to_csv(out_path, index=False)
    print(f"💾 Saved → {out_path}\n")


✅ transfermarket: 17 rows, 8 cols cleaned successfully.
💾 Saved → C:/Users/georg/Downloads/testings/working_files_to_cleaned\cleaned_final\transfermarket_cleaned.csv

✅ competitor_profiles: 15 rows, 3 cols cleaned successfully.
💾 Saved → C:/Users/georg/Downloads/testings/working_files_to_cleaned\cleaned_final\competitor_profiles_cleaned.csv

✅ news_topics: 423 rows, 8 cols cleaned successfully.
💾 Saved → C:/Users/georg/Downloads/testings/working_files_to_cleaned\cleaned_final\news_topics_cleaned.csv

✅ socials_topics: 2081 rows, 8 cols cleaned successfully.
💾 Saved → C:/Users/georg/Downloads/testings/working_files_to_cleaned\cleaned_final\socials_topics_cleaned.csv

✅ socials_overview: 233 rows, 8 cols cleaned successfully.
💾 Saved → C:/Users/georg/Downloads/testings/working_files_to_cleaned\cleaned_final\socials_overview_cleaned.csv

✅ google_headlines: 212 rows, 5 cols cleaned successfully.
💾 Saved → C:/Users/georg/Downloads/testings/working_files_to_cleaned\cleaned_final\google_head

In [19]:
# -----------------------------
# Summary
# -----------------------------
print("🎯 All PSV datasets cleaned successfully!")
print(f"📦 Total datasets cleaned: {len(cleaned_dataframes)}")
print("📁 Output folder: cleaned_final/")

🎯 All PSV datasets cleaned successfully!
📦 Total datasets cleaned: 22
📁 Output folder: cleaned_final/
